In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import kelp_param
import kelp3d_objs as k3
import ipyparallel as ipp
import concurrent.futures as cf
import itertools as it
#from sympy import divisors
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt
from scipy.interpolate import interp1d
import ipyvolume as ipv
import ipywidgets as ipw

import kelp_compute
import kelp_analyze

In [3]:
import sqlite3
import shutil
import os

### Define Constants

In [4]:
# Hard-coded parameters
rope_spacing = 20.0 # (m)
zmin, zmax = 0.0, 10.0 # (m)

# Light from above
I0 = 50.0 # (W/m^2)
phi_s = 0 # (rad)
theta_s = 0 # (rad)
decay = 0 # (?) - light (practically) only from directly above

# Calculated or otherwise boring parameters
xmin = -rope_spacing/2
xmax = rope_spacing/2
ymin = -rope_spacing/2
ymax = rope_spacing/2

# Set parameters
study_name = 'single'
a_water = 0.179
b = 0.219
kelp_dist='top-heavy'

base_dir = '../../results'

ns = 10
nz = 10
na = 10

num_scatters = 1
fd_flag = False
lis_options = "-i gmres -restart 100 -tol 1e-4"

# Compute
study_dir = os.path.join(base_dir, study_name)
data_dir = os.path.join(study_dir, 'data')

# TODO: Don't remove results?
shutil.rmtree(study_dir, ignore_errors=True)

os.makedirs(data_dir)
db_path = os.path.join(study_dir, '{}.db'.format(study_name))
conn = sqlite3.connect(db_path)
kelp_compute.create_table(conn, study_name)
conn.close()

In [6]:
kelp_compute.kelp_calculate(
    a_water, b,
    ns, nz, na,
    kelp_dist,
    num_scatters=0,
    fd_flag=True,
    lis_opts=lis_options,
    study_dir=study_dir,
    study_name=study_name
)

a_kelp = 1749.9999999999998
xmin = -5.0
xmax = 5.0
nx = 10
ymin = -5.0
ymax = 5.0
ny = 10
zmin = 0
zmax = 10
nz = 10
frond_lengths.shape = (10,)
frond_stds.shape = (10,)
num_fronds.shape = (10,)
water_speeds.shape = (10,)
water_angles.shape = (10,)
fs = 0.5
fr = 5.0
ft = 0.0004
p_kelp.shape = (10, 10, 10)
absorptance_kelp: 0.7 (<class 'float'>)
a_water: 0.179 (<class 'float'>)
b: 0.219 (<class 'float'>)
ns: 10 (<class 'int'>)
na: 10 (<class 'int'>)
nx: 10 (<class 'int'>)
ny: 10 (<class 'int'>)
nz: 10 (<class 'int'>)
ntheta: 10 (<class 'int'>)
nphi: 10 (<class 'int'>)
nomega: 82 (<class 'int'>)
num_dens: 120 (<class 'int'>)
kelp_dist: top-heavy (<class 'str'>)
CREATE STR VAR: ('kelp_dist', 'S9', 'kelp_dist_dim')
fs: 0.5 (<class 'float'>)
fr: 5.0 (<class 'float'>)
ft: 0.0004 (<class 'float'>)
max_length: 6.0 (<class 'float'>)
length_std: 1.2000000000000002 (<class 'float'>)
zmax: 10 (<class 'int'>)
rope_spacing: 10 (<class 'int'>)
I0: 50.0 (<class 'float'>)
phi_s: 0.0 (<class 'float'>)
t

In [7]:
for num_scatters in range(3):
    kelp_compute.kelp_calculate(
        a_water, b,
        ns, nz, na,
        kelp_dist, num_scatters,
        fd_flag=False,
        lis_opts=lis_options,
        study_dir=study_dir,
        study_name=study_name
    )

a_kelp = 1749.9999999999998
xmin = -5.0
xmax = 5.0
nx = 10
ymin = -5.0
ymax = 5.0
ny = 10
zmin = 0
zmax = 10
nz = 10
frond_lengths.shape = (10,)
frond_stds.shape = (10,)
num_fronds.shape = (10,)
water_speeds.shape = (10,)
water_angles.shape = (10,)
fs = 0.5
fr = 5.0
ft = 0.0004
p_kelp.shape = (10, 10, 10)
absorptance_kelp: 0.7 (<class 'float'>)
a_water: 0.179 (<class 'float'>)
b: 0.219 (<class 'float'>)
ns: 10 (<class 'int'>)
na: 10 (<class 'int'>)
nx: 10 (<class 'int'>)
ny: 10 (<class 'int'>)
nz: 10 (<class 'int'>)
ntheta: 10 (<class 'int'>)
nphi: 10 (<class 'int'>)
nomega: 82 (<class 'int'>)
num_dens: 120 (<class 'int'>)
kelp_dist: top-heavy (<class 'str'>)
CREATE STR VAR: ('kelp_dist', 'S9', 'kelp_dist_dim')
fs: 0.5 (<class 'float'>)
fr: 5.0 (<class 'float'>)
ft: 0.0004 (<class 'float'>)
max_length: 6.0 (<class 'float'>)
length_std: 1.2000000000000002 (<class 'float'>)
zmax: 10 (<class 'int'>)
rope_spacing: 10 (<class 'int'>)
I0: 50.0 (<class 'float'>)
phi_s: 0.0 (<class 'float'>)
t

In [8]:
# Retrieve

conn = sqlite3.connect(db_path)
res = kelp_analyze.query_results(conn, study_name, ns=ns, nz=nz, na=na)[0]
conn.close()
irrad = res['irrad'][:]

print("{:.2f}% nan".format(np.sum(np.isnan(irrad))/np.size(irrad)))

# Visualize

ipv.figure()
ipv.volshow(irrad)
ipv.show()

IndexError: list index out of range